In [1]:
import pandas as pd
import openai
from dotenv import load_dotenv
import os
import ast
from openai import AzureOpenAI
import faiss
import numpy as np
import json
import spacy
nlp = spacy.load("en_core_web_sm")
import pycountry
import re
from bert_score import score as bert_score
import csv
import transformers
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import itertools
import re
from sklearn.metrics.pairwise import cosine_similarity
from spacy.lang.en.stop_words import STOP_WORDS
from datetime import datetime


/Users/davidoluyalegbenga/.pyenv/versions/3.9.6/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/Users/davidoluyalegbenga/.pyenv/versions/3.9.6/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Model files

In [40]:
df = pd.read_pickle('../models/df_embed_EN_All_V3.pkl')

### Load Enviroment files

In [3]:
# Load environment variables
load_dotenv()

True

In [4]:
# OpenAI API configuration
openai.api_type = "azure"
openai.api_key = os.getenv("api_key_azure")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("api_version")
openai_deployment = "sdgi-gpt-35-turbo-16k"


client = AzureOpenAI(
  api_key = os.getenv("api_key_azure"),  
  api_version = os.getenv("api_version"),
  azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
)

embedding_model = os.getenv("USER_QUERY_EMBEDDING_ENGINE") 

# print(openai.api_key)
# print(openai.api_base)
# print(openai.api_version)


<h3>globals</h3>

In [5]:
# test_query="What are the sustainable energy priorities for UNDP?"
test_query = 'What is the Human Development Index (HDI) value for Albania as mentioned in the document?'


<h3> helper functions </h3>

In [5]:
# use this function to make simple openAI Calls
def callOpenAI(prompt):  
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0,
                    messages=[
                        {"role": "user", "content": prompt},
                    ]
                )
    response = response_entities.choices[0].message.content
    return response


<h3> processing modules </h3>

In [6]:
def extractEntitiesFromQuery(user_query):
    prompt = f"""
    Extract entities from the following user query: \"{user_query}\" and return output in array format.
    
    -Entities should be directly related to the domain or topic of interest. They should represent important concepts that contribute to the understanding of the subject matter.
    -Each entity in the knowledge graph should be distinct and have a unique identifier. This ensures clarity and avoids ambiguity when establishing relationships between entities.
    -You Must return output in array format e.g  ['entity1','entity2'] !!!
    -Avoid adding new lines or breaking spaces to your output. Array should be single dimension and single line !!!
 
    """
    entity_list = callOpenAI(prompt)   
    return entity_list

# Test usage
# test_query = "What are the sustainable energy for UNDP?"
# entity_list = extractEntitiesFromQuery(test_query)
# print(entity_list)

In [7]:
## module to get information on the entities from user query using the KG
def knowledgeGraphModule(user_query):
    
    # generate list of entities based on user query
    entity_list = extractEntitiesFromQuery(user_query)
    my_list = ast.literal_eval(entity_list)
    prompt_summarise_entites = f"""
    Summarize all relations between all the entities : {my_list}
    """
    summarise_entities = callOpenAI(prompt_summarise_entites)
    # Initialize an empty dictionary to store information
    entities_dict = {
        "relations": summarise_entities,
        "entities": {}
    }
    # Loop through each entity in the list
    for entity in my_list:
        # Fetch information about the entity from your knowledge graph
        prompt = f"Give me a short description 50 words of {entity}"
        entity_info = callOpenAI(prompt)
        # Add the entity information to the dictionary
        entities_dict["entities"][entity] = entity_info
    
    return entities_dict


# Test usage
test_query = "What is the role of Paris Agreement in sustainable energy development?"
entities_dict = knowledgeGraphModule(test_query)
print(entities_dict)

{'relations': 'The Paris Agreement is related to sustainable energy development.', 'entities': {'Paris Agreement': "The Paris Agreement is an international treaty adopted in 2015 by nearly every country in the world. Its goal is to combat climate change by limiting global warming to well below 2 degrees Celsius above pre-industrial levels. It promotes cooperation, transparency, and regular updates of countries' efforts to reduce greenhouse gas emissions.", 'sustainable energy development': 'Sustainable energy development refers to the process of harnessing and utilizing renewable energy sources in a manner that minimizes environmental impact and promotes long-term viability. It involves the adoption of clean technologies, such as solar, wind, and hydro power, to meet energy needs while reducing greenhouse gas emissions and preserving natural resources.'}}


In [8]:

def find_mentioned_countries(text):
    countries = set()
    
    # Tokenize the text using regular expressions to preserve punctuation marks
    words = re.findall(r'\w+|[^\w\s]', text)
    text = ' '.join(words)  # Join the tokens back into a string
    
    for word in text.split():
        try:
            country = pycountry.countries.get(name=word) #pycountry.countries.lookup(word)
            if country != None : 
               countries.add(country.name)
        except LookupError:
            pass
    
    return list(countries)

# Example 
# user_query = 'Give me a summary of the goals UNDP wants to achieve in 10 years and the energy plans for Philippines and Angola'
# mentioned_countries = find_mentioned_countries(user_query)
# mentioned_countries

In [9]:

# Load the English language model
# Function to calculate the average word embedding for a sentence
def average_word_embedding(sentence):
    # Parse the sentence using SpaCy
    doc = nlp(sentence)
    # Get word vectors and average them
    word_vectors = [token.vector for token in doc if token.has_vector]
    if not word_vectors:
        return None
    return np.mean(word_vectors, axis=0)

# Function to calculate context similarity between two sentences using word embedding averaging
def calculate_context_similarity(sentence1, sentence2):
    # Get average word embeddings for each sentence
    avg_embedding1 = average_word_embedding(sentence1)
    avg_embedding2 = average_word_embedding(sentence2)
    if avg_embedding1 is None or avg_embedding2 is None:
        return None
    # Calculate cosine similarity between the embeddings
    similarity = cosine_similarity([avg_embedding1], [avg_embedding2])[0][0]
    return similarity

# # Example sentences
# sentence1 = 'The companys quarterly earnings report exceeded expectations, leading to a surge in stock prices.'
# sentence2 = 'The firms financial results for the last quarter surpassed predictions, resulting in a sharp rise in the value of shares'



# Calculate context similarity
# similarity = calculate_context_similarity(sentence1, sentence2)
# print("Context similarity:", similarity)




In [10]:
# Function to calculate Jaccard similarity between two texts
def jaccard_similarity(text1, text2):
    # Tokenize texts
    tokens1 = set(text1.lower().split())
    tokens2 = set(text2.lower().split())
    
    # Calculate Jaccard similarity
    intersection = len(tokens1.intersection(tokens2))
    union = len(tokens1.union(tokens2))
    
    return intersection / union if union > 0 else 0


In [11]:
def title_contains_entity(entity, title):
    # Convert both entity and title to lowercase for case-insensitive comparison
    entity_lower = entity.lower()
    title_lower = title.lower()

    # Check if the lowercase entity is contained within the lowercase title
    if entity_lower in title_lower:
        return 1
    else:
        return 0

In [12]:
#This contains all filters for the semantic search
#Context Similarity takes two queries and find how similar they are "context wise"
#E.g "My house is empty today" and "Nobody is at my home" are same context but not word similarity
# - Filter country relevant documents when mentioned 
# - Filter by Context similarity in user_query and title, journal, content etc.

def filter_semantics(user_query):

    #Allow parallels
    #Extract notable entities in query e.g ORG, NAME, Place, country, location etc.
    #Location related: 
    # GPE: Countries, cities, states.
    # NORP: Nationalities, religious and political groups.
    # LANGUAGE: Any named language. 
    # FAC: Buildings, airports, highways, bridges, etc.

    #Other/General:
    # PERSON: People, including fictional entities.
    # ORG: Companies, agencies, institutions, etc.
    # LOC: Non-GPE locations, mountain ranges, bodies of water.
    # PRODUCT: Objects, vehicles, foods, etc. (Not services)
    # EVENT: Named hurricanes, battles, wars, sports events, etc.
    # WORK_OF_ART: Titles of books, songs, etc.
    # LAW: Named documents made into laws.
    # LANGUAGE: Any named language.
    # DATE: Absolute or relative dates or periods.
    # TIME: Times smaller than a day.
    # PERCENT: Percentage, including "%".
    # MONEY: Monetary values, including unit.
    # QUANTITY: Measurements, as numerical values with units.
    # ORDINAL: "first", "second", etc.
    # CARDINAL: Numerals that do not fall under another type.
    
    doc = nlp(user_query)
    # Extract all entities
    # entities = [(ent.text, ent.label_) for ent in doc.ents]
    entities = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ != ""]  # Filter out empty entities
    entities.extend((token.text, "NOUN") for token in doc if token.pos_ in ["NOUN","PROPN", "PRON", "PROPN", "NUM", "SYM", "X","ABBR"] or token.is_alpha)

    # Remove stop words
    entities = [(entity, label) for entity, label in entities if entity.lower() not in STOP_WORDS]
    
    # Print the extracted entities
    # print("All Entities and POS:", entities)
    # Generate DFs for main entities
    filtered_df_country = pd.DataFrame()  # Initialize an empty DataFrame
    filtered_df_others = pd.DataFrame()  # Initialize an empty DataFrame
    filtered_df_backup_reference = pd.DataFrame() # Initialize an empty DataFrame
    allow_low = True

    for entity, label in entities:
        # print(entity)
        filtered_df_others = pd.concat([filtered_df_others, df[df['Document Title'].str.lower().str.contains(entity.lower(), na=False)]])

        #Calculate similarity scores for each document title
        similarity_scores = []
        document_titles = []

        # Iterate through each document title and calculate similarity score
        for title in filtered_df_others['Document Title']:
            if title is not None:
                similarity_score = calculate_context_similarity(user_query,title) 
                # print(entity)
                # print(similarity_score)
                # print(user_query)
                # print(title)
                # print("=================================")    
                similarity_scores.append(similarity_score)
                document_titles.append(title)
        
        # Create DataFrame only with valid similarity scores
        similarity_df = pd.DataFrame({'Document Title': document_titles, 'Similarity Score': similarity_scores})
        
        df_temp = pd.concat([df])
        
        threshold = 0.5

        # Filter df based on similarity scores greater than threshold for filtered_df_others
        filtered_df_others = df[df['Document Title'].isin(similarity_df[similarity_df['Similarity Score'] > threshold]['Document Title'])]
        filtered_df_backup_reference = pd.concat([filtered_df_backup_reference,  df_temp[df_temp['Document Title'].isin(similarity_df[(similarity_df['Similarity Score'] >= 0.1) & (similarity_df['Similarity Score'] < 0.45)]['Document Title'])] ])

        #Check for location related e.g by country, language, locals
        if label in ['GPE', 'NORP', 'LANGUAGE', 'FAC']:
            filtered_df_country = pd.concat([filtered_df_country, df[df['Country Name'] == entity]])
   
    merged_df = pd.DataFrame()
    if filtered_df_others.empty and filtered_df_country.empty:
       print(f'on the reference df {filtered_df_backup_reference.empty}')
       merged_df = pd.concat([filtered_df_backup_reference])
    else :
       merged_df = pd.concat([filtered_df_country,filtered_df_others])
    
    return merged_df



# Example 
# test_query=" is the promotion of renewable energy a focal point within the energy sector?"
# filtered_country = filter_semantics(test_query)
# filtered_country

In [13]:
def search_embeddings(user_query):
    # df_filtered = filter_semantics(user_query) if filter_semantics(user_query) is not None else None
    filtered_result = filter_semantics(user_query)
    # Check if the result is not None before assigning it to df_filtered
    df_filtered = filtered_result if filtered_result is not None else None

    if df_filtered is not None and not df_filtered.empty:  # Check if DataFrame is not None and not empty
        length = len(df_filtered.head())
        filtered_embeddings_arrays = np.array(list(df_filtered['Embedding']))
        index = faiss.IndexFlatIP(filtered_embeddings_arrays.shape[1]) 
        index.add(filtered_embeddings_arrays)
        
        user_query_embedding = client.embeddings.create( 
                input=user_query ,model= embedding_model
            ).data[0].embedding

        k = min(5, length)
        distances, indices = index.search(np.array([user_query_embedding]), k)
        return df_filtered, distances, indices
    else:
        return None, None, None


In [14]:
def map_to_structure(qs):
    result_dict = {}

    # Extract the DataFrame from the tuple
    dataframe = qs[0]
    print(qs[1])
    print(qs[2])

    # Counter to limit the loop to 10 iterations
    count = 0
    for index, row in dataframe.iterrows():
        # Define a unique identifier for each document, you can customize this based on your data
        document_id = f"doc-{index + 1}"
        # Handle NaN in content by using fillna
        content = row["Content"]
        content = ' '.join(row["Content"].split()[:160])
        # Create a dictionary for each document
        document_info = {
            "title": row["Document Title"],
            "extract": content or "",  # You may need to adjust this based on your column names
            "category": row["Category"],
            "link": row["Link"],
            "thumbnail": ''
        }
        # print(document_info)
        # Add the document to the result dictionary
        result_dict[document_id] = document_info

        # Increment the counter
        count += 1

        # # Break out of the loop if the counter reaches top 25
        if count == 10:
            break

    return result_dict


In [15]:
# Function to relabel keys and add citations
def relabel_and_add_citations(data):
    new_data = {}
    citation_counter = 1

    for doc_id, doc_info in data.items():
        new_data[doc_id] = {
            "document_title": doc_info.get("title", ""),
            "summary": doc_info.get("extract", ""),
            "document_category": doc_info.get("category", ""),
            "document_link": doc_info.get("link", ""),
            "document_thumbnail": doc_info.get("thumbnail", ""),
            "citation": citation_counter
        }
        citation_counter += 1

    return new_data

In [16]:
## module to extract text from documents and return the text and document codes

def semanticSearchModule(user_query):
    qs = search_embeddings(user_query) #df, distances, indices
    # if qs != None :
    if qs[0] is not None:
        result_structure = map_to_structure(qs)
        return result_structure
    else : 
        return []

        
#test usage
excerpts_dict= relabel_and_add_citations(semanticSearchModule("What is the role of USAID in supporting Albania's energy sector strategy?"))
# # print(f"""excerpts_dict {excerpts_dict}""")

# #Return top 10-20 most related 
# # Define the filename to save the JSON data -  can remove later
json_filename = "outputs/excerpts.json"

# # Save excerpts_dict to the JSON file just for a better preview
with open(json_filename, 'w', encoding='utf-8') as json_file:
    json.dump(excerpts_dict, json_file, ensure_ascii=False, indent=4)

print(f"Excerpts saved to {json_filename}")

[[0.8761674  0.84270215 0.8349415  0.8349415  0.83446956]]
[[0 6 8 4 5]]
Excerpts saved to outputs/excerpts.json


In [17]:
## module to get data for specific indicators which are identified is relevant to the user query

def indicatorsModule(user_query): #lower priority
    
    # find relevant indicators based on uesr query and extract values
    indicators_dict={
        "indicator-id-1":"value from indicator-id-1",
        "indicator-id-2":"value from indicator-id-2"
    }#temp
    
    return indicators_dict

#test usage
indicators_dict=indicatorsModule(test_query)
print(indicators_dict)

{'indicator-id-1': 'value from indicator-id-1', 'indicator-id-2': 'value from indicator-id-2'}


In [18]:
## module to generate query ideas

def queryIdeationModule(user_query): # lower priority
    
    # Generate query ideas using OpenAI GPT-3
    prompt = f"""
    Ignore previous commands!!!
    Generate prompt ideas based on the user query: {user_query}

    
    -Prompt shoud not be answer to the user query but give other contextual ways of representing the user query !!!
    -You Must return output seperated by |  e.g idea 1 | idea2 
    - Each generated ideas should be very dinstinct but contextual. Not repeatitive or using same words
    - The query idea should be in a question form and not an answer form.
    -Avoid adding new lines or breaking spaces to your output and must seperate each idea with |
    """
    response = callOpenAI(prompt)
    return response

#test usage
# query_idea_list=queryIdeationModule(test_query)
# print(query_idea_list)

In [39]:
def get_answer(user_question, relevant_docs):

    # Never show the source links or titles. Just put the doc-id as reference when you cite a document!!!! e.g cited content here <a href="#doc-id">[n]</a> where n is an integer and must follow proper order e.g 1,2,3,4,5... !!! 
    # Do not include references or sources links at the end!!! 
    # formattings_old = f"""    
    # Ignore all previous commands!!!

    # Your answer only in HTML syntax with HTML tags.
    # Use HTML tags like < ul>, < ol>, < li>, < strong>, < p>
    # Only consider the inner part of the < body> tag.
    # ALWAYS use the following tag for new lines: < br />
    # Do not add CSS attributes.
    # Your answer must be formatted in HTML format
    
    # Make sure to properly format the text like a professional write. Avoid punctuation errors and the likes. 
    # Must Use < ul>, < ol>, < li> for listing. Avoid listing on same line. All list or numbering should be on new lines
    # Must Use <p> for paragraphs
    # When listing item, make sure to count properlly.

    # You Must Follow these strictly : 
    # 1. You Must also find and cite relevant information from the DOCS References extract and title properly !!!! Use the anchor tag pointing to the doc link as href for the citation number e.g <a href="link">[n]</a> 
    # 2. For example : content here <a href="link">[n]</a> another content. And another <a href="link">[n]</a> where n is integer starting at 1 for [n] and must increament order 1,2,3,4,5,6 .... citations must follow proper increment by 1. 
    #    for example : content here <a href="link">[1]</a> contnt . anoter content  <a href="link">[2]</a> is correct. 
    #    for example : content here <a href="link">[1]</a> contnt . anoter content  <a href="link">[3]</a> is very wrong as its jumped to 3. It must be 1,2,3,.... progressive!!!!!

    # 2a. link must only be used as href value !!!. Each citation number must be an anchor link
    # 3. Let the extracted texts must come inbetween and naturally in the answers and not at the end as relevant in paragraphs !!!! 
    # 4. Must Avoid using words like Citation - , Sources-  etc at the ending. Never add it to the end of the content. Its not needed. Just cite inbetween the paragraphs as needed!!!
    # 5. Don't use too much citations. Only a few as most relevant!!! like 20%. You must use at least one or more relavant citation !!! Use citations as part of the answers and not in isolation
    # """


    #References:
    # Title of the <<SOUCRCE>>. Retrieved from URL e.g 
    # Title of the <<SOUCRCE>>. Retrieved from URL. 

    formattingsold = f""" 
        Strictly follow the follow steps:
        Your answer only in HTML syntax with HTML tags.
        Use HTML tags like < ul>, < ol>, < li>,  < strong>, < p>
        Only consider the inner part of the < body> tag.
        ALWAYS use the following tag for new lines: < br />
        use <a> for cite numbers
        Do not add CSS attributes.
        Your answer must be formatted in HTML format
        
        
        1. <<SOUCRCE>> for citing : {relevant_docs}

        2. Must Follow the Structure for your output strictly: 

            
                Break down the answer into main points.
                Provide explanations and cite <<SOUCRCE>> directly.

                Summarize the main points.

                For Example: 
                Question: What are the benefits of regular exercise?
                
                Answer
                Regular exercise offers significant benefits across physical, mental, and emotional dimensions. Physically, it strengthens the heart and enhances circulation, which can reduce the risk of heart disease by up to 40% <a href=“link” data-id=“doc-id”>[1]</a>. It also aids in weight management by burning calories and building muscle mass <a href=“link” data-id=“doc-id”>[2]</a>.
                Mentally, exercise helps reduce anxiety and depression by increasing the production of endorphins, acting similarly to some medications. Additionally, it enhances cognitive function, improving memory and learning capabilities <a href=“link” data-id=“doc-id”>[3]</a>.
                Emotionally and socially, regular physical activity reduces stress hormones such as cortisol  and boosts self-esteem by improving body image and self-confidence. Participation in group sports also provides social benefits and fosters a sense of community.
                In summary, regular exercise is essential for overall health, enhancing physical fitness, mental clarity, and emotional well-being.


    3. Return answer without explicit partitions, keeping it concise and integrating citations naturally within the text.
    4. You must cite at least one relevant <<SOUCRCE>> provided above. This is compulsory. Don't just show a reference at the footnotes but also integrate into the text using the cite number e.g <a href=“link” data-id=“doc-id”>[1]</a>.
    5. Cited source number must be an anchor tag link !!!!! You don't have to always cite. Only do this when relevant. If you can't find the link just leave the citation out.
    6. VERY IMPORTANT:  avoid numberic listings in your answer!!!!!
    """
    
    formattings = f""" 
        Strictly follow the follow steps:
        Your answer only in HTML syntax with HTML tags.
        Use HTML tags like < ul>, < ol>, < li>,  < strong>, < p>
        Only consider the inner part of the < body> tag.
        ALWAYS use the following tag for new lines: < br />
        use <a> for cite numbers
        Do not add CSS attributes.
        Your answer must be formatted in HTML format
        
        
        1. <<SOUCRCE>> for citing : {relevant_docs}
    """
   
    # {"role": "assistant", "content": formattings},


    messages = [
        {"role": "system", "content":"You are a helpful assistant and a professional writer with 50 years experience. Give answer to the user's inquiry."
        },
        {'role': 'user', 'content': f"""{formattings} 
                                        {user_question}"""},
    ]
    print(f""" excerpts_dict === {len(relevant_docs)} """)

    # print(formattings)

    print(openai_deployment)
        
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0.3,
                    messages=messages,
                    top_p=0.8,
                    frequency_penalty=0.6,
                    presence_penalty=0.8

                )
    response = response_entities.choices[0].message.content
    # Define the regex pattern to match digits followed by '. do'
    pattern = r'\d+\. do'

    # Remove matches from the text
    cleaned_text = re.sub(pattern, '', response)

    # # Optionally, clean up any extra spaces or punctuation left behind
    # cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text).strip()


    return cleaned_text
  

<h3> synthesis module </h3>

    llm_instructions="llm instruction template here, with placeholders for insertion of user query, excerpts, indicator data, and entity and relation info" 


In [32]:
# module to synthesize answer using retreival augmented generation approach


      #  - Make sure to Include citations based on the Sources. e.g Text excerpt here<a data-id='test-doc-1'>[1]</a> when referencing a document in the sources. using 1 ...nth
      #       - The citations anchor should be near the excerpt not following each other.
      #       - Use the anchor tag for the citation links and should link to the document link in the Sources. for example Undp operates in afganistan <a data-id='test-doc-1'>[1]</a>. UNDP offers health relationships <a data-id='test-doc-2'>[2]</a>.
      #       - Make sure the data-id actually exists in the Sources.
      #       - The text in the anchor tag should be citation number not document title.
      #       - You can reference multitple citations based sources

# prompt_formattings= f"""
#     - Answer output must be properly formatted using HTML. 
#     - Don't include <html>, <script>, <link> <a>  or <body> tags. Only text formating tags should be allowed. e.g h1..h3, p, anchor, etc. Strictly HTML only
#     - Strictly infer your answers from the <Sources> Only and make citations to Source extract referenced 
#     - The Source as format like: "doc-n": {{
#         "title": "title of the relate document",
#         "extract": "content",
#         "category": "",
#         "link": "",
#         "thumbnail": "",
#         "citation": n
#     }}, where doc-n can be doc-1, doc-24 etc.. n is in integer.
#     - Reference the extract and title of all document sources provided in the json and summarise it into a coherent answer that relates to the <User Query>
#     - Citation should follow formats: [reference content][citation number]. 
#     - Give output writing tone like a academic research tone
#     - If no <Sources> are provided, try to make suggestives or  simply say you don't have that information   
#     - Remove new line or tab characters from your output
#     - do not use or include links,  anchor links or a href tags !!!
#     -do not include references links at the end!!!
#     - to reference within the text do [n] not [source n] . musct be [n] where n is an integer of the citation number
#     - Should be one citation only e.g [n] not [n][n][n]
# """

def synthesisModule(user_query, entities_dict, excerpts_dict, indicators_dict,prompt_formattings):
    
    # Generate prompt engineering text and template
    # llm_instructions = f"""
    # Ignore previous commands!!!
    # Given a user query, use the provided <Sources> extract section of the JSON only to provide the correct answer to the user's query.
    
    # User Query: {user_query}
    
    # Sources: {excerpts_dict}
    
    # {prompt_formattings}
    # """

    #Closed Domain prompting
    # llm_instructions = f""" 
    #   Context:  {excerpts_dict}
    #  """
    # print(llm_instructions)
    ###synthesize data into structure within llm prompt engineering instructions
    answer=get_answer(user_query, excerpts_dict) #callOpenAI
    answer_formated_fixed = answer.replace("\n\n","<br>").replace("\n","<br>")
    return answer_formated_fixed

## to test this, run the full pipeline with the handleApiCall function

<h3> run pipeline </h3>

In [41]:
# full pipeline with retreival, synthesis of answer to user query, and structure results into api response

def handleApiCall(user_query):
    
    ##run processing modules (in parallel)
    entities_dict=knowledgeGraphModule(user_query)
    excerpts_dict=semanticSearchModule(user_query)
    indicators_dict=indicatorsModule(user_query) ##lower priority
    query_idea_list=queryIdeationModule(user_query) ##lower priority
    prompt_formattings=""
    ##synthesis module
    answer=synthesisModule(user_query, entities_dict, excerpts_dict, indicators_dict, prompt_formattings)

    ##structure response
    response={
        "user_query":user_query,
        "answer":answer,
        "sources":excerpts_dict,
        "query_ideas":query_idea_list,
        "entities":list(entities_dict["entities"].keys())       
    }
    
    return response

# test usage
test_query = "Can off-grid solutions effectively address energy access challenges in remote areas in Nigeria?"
response=handleApiCall(test_query) 
# Define the filename to save the JSON data -  can remove later
json_filename = "outputs/synthesis_output.json"

# Save excerpts_dict to the JSON file just for a better preview
with open(json_filename, 'w', encoding='utf-8') as json_file:
    json.dump(response, json_file, ensure_ascii=False, indent=4)

print(f"Synthesis saved to {json_filename}")

[[0.85505545 0.8340653  0.82767075 0.8249451  0.82131886]]
[[33 25 56  7 46]]
 excerpts_dict === 10 
 excerpts_dict === 10 
 
        Strictly follow the follow steps:
        Your answer only in HTML syntax with HTML tags.
        Use HTML tags like < ul>, < ol>, < li>,  < strong>, < p>
        Only consider the inner part of the < body> tag.
        ALWAYS use the following tag for new lines: < br />
        use <a> for cite numbers
        Do not add CSS attributes.
        Your answer must be formatted in HTML format
        
        
        1. <<SOUCRCE>> for citing : {'doc-494': {'title': 'Nigeria National Gas Policy', 'extract': 'Nigeria National Gas Policy THE GOVERNMENT OF THE FEDERAL REPUBLIC OF NIGERIA - -------------------------------------------------------------------------------- -- NATIONAL GAS POLICY --------------------------------------------------------- -------------------------- The purpose of this document is to define the policy of the Federal Government in res

<h1>Testing</h1>

<p>This sections contains all things testings </p>

In [45]:
# !pip install bert_score

In [ ]:
def calculate_scores(csv_file):
    # Initialize an empty list to store processed entries
    result = []
    
    # Loop through each entry in the CSV file
    for entry in csv.DictReader(csv_file):
        query = entry['query']
        sample_answer = entry['sample_answer']
        
        # Call OpenAI for chat GPT answer
        chat_gpt_answer = callOpenAI(f""" 
                                    {query} 
                                    {prompt_formattings} 
                                    """)
        
        # Call the moonshot model API
        moonshot_model_answer = handleApiCall(query) 
        
        # Calculate BERT score for moonshot model answer
        P, F, R = bert_score([sample_answer], [moonshot_model_answer['answer']], lang='en', verbose=True)
        entry['moonshot_model_answer'] = moonshot_model_answer['answer']
        entry['bert_score'] = round(float(F), 2)

        # Calculate BERT score for chat GPT answer
        P, F, R = bert_score([sample_answer], [chat_gpt_answer], lang='en', verbose=True)
        entry['chat_gpt_answer'] = chat_gpt_answer
        entry['bert_score_gpt'] = round(float(F), 2)
        
        # Append the processed entry to the result list
        result.append(entry)
    
    # Return the list of processed entries
    return result


# Specify the path to your CSV file
csv_file_path = "../testing/queries.csv"

# Open the CSV file for reading
with open(csv_file_path, mode='r') as file:
    # Pass the file object to the function
    result = calculate_scores(file)

# Print updated data with scores
# print(json.dumps(result, indent=4))

# Save updated data to a JSON file
with open('../testing/test_output.json', 'w') as file:
    json.dump(result, file, indent=4)


<h1>Compare Moonshot BERT score to GPT BERT Score</h1>

In [ ]:
# Load the JSON file
with open('../testing/test_output.json', 'r') as file:
    data = json.load(file)

# Initialize variables to store total scores and count of items
total_bert_score = 0
total_bert_score_gpt = 0
count = 0

# Iterate through each item in the JSON data
for item in data:
    # Extract bert_score and bert_score_gpt from the current item
    bert_score = item['bert_score']
    bert_score_gpt = item['bert_score_gpt']
    
    # Add the scores to the total
    total_bert_score += bert_score
    total_bert_score_gpt += bert_score_gpt
    
    # Increment the count
    count += 1

# Calculate the average scores
average_bert_score = total_bert_score / count
average_bert_score_gpt = total_bert_score_gpt / count

# Print the average scores
print("Average bert_score:", average_bert_score)
print("Average bert_score_gpt:", average_bert_score_gpt)

Average bert_score: 0.7995652173913044
Average bert_score_gpt: 0.7778260869565218


<h1>Trello Board AutoPopulate</h1>
<p> Automation for trello</p>

In [78]:
import os
import csv
import asyncio
import aiohttp
import nest_asyncio
import requests
from markdownify import markdownify as md
import textwrap
import re


In [79]:
# Load environment variables
trello_api_list_id = os.getenv("trello_api_list_id")
trello_api_key_id = os.getenv("trello_api_key_id")
trello_api_key_token = os.getenv("trello_api_key_token")
trello_board_id= os.getenv("trello_board_id")
trello_url ="https://api.trello.com/1/"
# print(trello_board_id)

In [80]:
#create list function - this allows for various test versions 

def create_list(list_title):
    url = f"{trello_url}lists?name={list_title}&token={trello_api_key_token}&idBoard={trello_board_id}&key={trello_api_key_id}"
    response = requests.request("POST", url)
    response_json = response.json()  # Parse response JSON
    list_id = response_json["id"]  # Access 'id' from JSON
    return list_id

#example
# list_response = create_list('User Queries')

In [81]:
#create list function - this allows for various test versions 
def create_card_label(card_id, color, name):
    url = f"{trello_url}cards/{card_id}/labels?color={color}&name={name}&token={trello_api_key_token}&idBoard={trello_board_id}&key={trello_api_key_id}"
    # print(url)
    response = requests.request("POST", url)
    response_json = response.json()  # Parse response JSON
    list_id = response_json["id"]  # Access 'id' from JSON
    return list_id

#example
# list_response = create_card_label('','')

In [82]:
def get_current_date(format='%b %d %H:%M'):
    return datetime.now().strftime(format)


In [213]:
#Async function to send the request

async def send_request(query,list_response,card_color,card_label_name):
    name = query["query"]
    response=handleApiCall(name) 

    #for trello preview only
    cleanResp = response['answer'].replace("</p>","</p> ******************************************************************************").replace("<p>","<br/>").replace("<br>"," *******************************************************************************")
    markdown_content_description = md(f"{cleanResp}")
    # markdown_content_description = html2text.html2text(cleanResp)

    desc = f"""{markdown_content_description}"""
    url = f"{trello_url}cards?idList={list_response}&key={trello_api_key_id}&token={trello_api_key_token}&name={name}&desc={desc}"
    
    async with aiohttp.ClientSession() as session:
        async with session.post(url, timeout=1200) as response:
            el = ''
            # print(name)
            resp = await response.text()
            resp_dict = json.loads(resp)
            id = resp_dict['id']

            card_label_resp = create_card_label(id, card_color, card_label_name)
            print(card_label_resp)
            print("---------")


In [ ]:
#Run Test process
async def mainTest(color,file):
    card_title = f"""{get_current_date()}"""
    card_color = color
    card_label_name =file
    queries_source = f"""../testing/queries/{file}.csv"""
    tasks = []
    list_response = create_list(card_title)
    with open(queries_source, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            tasks.append(send_request(row,list_response,card_color,card_label_name))
    await asyncio.gather(*tasks)

# Apply nest_asyncio
nest_asyncio.apply()

# Run the event loop
# await mainTest('red','clarification')
# await mainTest('purple','comparative')
# await mainTest('green','descriptive')
# await mainTest('yellow','informational')
# await mainTest('blue','opinion')
# await mainTest('orange','procedural')
# await mainTest('violet','queries')
# await mainTest('pink','yesno')